**Problem set 1.2 IO: Production Function Estimation OP and ACF**

Author: Jordi Torres 

Start Date: 11/12/2025



In [179]:
using StatFiles, DataFrames

#Input path
data_path = "/Users/jorditorresvallverdu/Library/Mobile Documents/com~apple~CloudDocs/tse/year2/term1/io/part2/GMdata.dta" ##-->CHANGE JUST THIS PART MANUALLY!

#Output path
export_dir = "/Users/jorditorresvallverdu/Documents/GitHub/TSE-2025-2026/IO/" ##Change this manually too

main = DataFrame(load(data_path))



Row,index,sic3,yr,ldsal,lemp,ldnpt,ldrst,ldrnd,ldinv
,Int16?,Int16?,Int8?,Float32?,Float32?,Float32?,Float32?,Float32?,Float32?
1,1,374,73,7.01896,2.51098,7.04537,3.7509,2.03022,4.98944
2,1,374,78,7.09394,2.67104,7.18584,3.79655,2.04228,4.90343
3,1,374,83,6.4082,2.00647,7.4539,3.92492,2.0334,3.82753
4,2,366,78,3.52628,-0.0523465,2.4056,2.61583,1.12504,0.633175
5,2,366,83,4.37366,0.640801,3.4784,3.22696,1.77073,1.91537
6,2,366,88,5.19584,0.518794,3.46187,3.99812,2.82213,2.59
7,3,358,78,3.80885,-0.543005,1.72808,0.746074,-1.05687,-0.748376
8,3,358,83,3.85115,-0.478036,2.69638,1.32711,0.265216,0.725182
9,4,381,73,4.25076,0.210261,3.03931,2.75336,0.812571,2.16101


In [180]:
##1.Ols on the full panel

df= dropmissing(main, [:ldsal, :lemp, :ldnpt]) ##no missings


X= hcat(
    ones(length(df.ldsal)),
    df.ldnpt,
    df.lemp
)

beta_ols= (X'X)\(X'df.ldsal)


3-element Vector{Float64}:
 3.0798222249143237
 0.42719629387166963
 0.5433117916971737

In [181]:
using FixedEffectModels, RegressionTables


reg_ols1 = reg(
    df,
    @formula(ldsal ~ ldnpt + lemp),
    Vcov.cluster(:index)
)

regtable(
    reg_ols1;
    renderSettings = LatexTable(),
    file = joinpath(export_dir, "reg_ols1.tex")
)

┌ Warning: renderSettings is deprecated. Specify render type with render=LatexTable() and file with file=/Users/jorditorresvallverdu/Documents/GitHub/TSE-2025-2026/IO/reg_ols1.tex
└ @ RegressionTables /Users/jorditorresvallverdu/.julia/packages/RegressionTables/wzLBu/src/regtable.jl:441


\begin{tabular}{lr}
\toprule
            & \multicolumn{1}{c}{ldsal} \\ 
\midrule
(Intercept) &                  3.080*** \\ 
            &                   (0.061) \\ 
ldnpt       &                  0.427*** \\ 
            &                   (0.019) \\ 
lemp        &                  0.543*** \\ 
            &                   (0.024) \\ 
\midrule
$N$         &                     2,971 \\ 
$R^2$       &                     0.919 \\ 
\bottomrule
\end{tabular}


In [182]:
#we define a dataset where only the firms appearing in all years stay. Balanced.

using DataFrames


df1 = dropmissing(df, :yr)
T   = length(unique(df1.yr))

good = combine(groupby(df1, :index)) do sdf
    balanced=
    all(.!ismissing.(sdf.ldsal)) &&
    length(unique(sdf.yr)) == T
    (; keep = balanced)
end

filter_k= good[good.keep, :index]
df_balanced = df1[in.(df1.index, Ref(filter_k)), :]


Row,index,sic3,yr,ldsal,lemp,ldnpt,ldrst,ldrnd,ldinv
,Int16?,Int16?,Int8,Float32,Float32,Float32,Float32?,Float32?,Float32?
1,8,355,73,7.30756,3.157,5.67673,4.4641,2.4318,4.5587
2,8,355,78,7.04852,2.91777,5.61488,4.31813,2.59354,3.82598
3,8,355,83,6.47044,1.89462,4.90529,4.5393,1.73744,2.56534
4,8,355,88,6.67208,2.02009,5.25774,4.52065,2.51504,2.5724
5,13,283,73,7.37242,3.05965,6.65504,6.0612,4.49305,4.47722
6,13,283,78,7.87958,3.29595,6.77531,6.35875,4.82749,4.83307
7,13,283,83,8.17725,3.53596,7.23209,6.61925,5.33549,5.75958
8,13,283,88,8.44212,3.65716,7.5052,7.21734,6.0851,6.22851
9,16,280,73,4.1332,-1.60944,1.99029,0.951125,-1.21886,-0.731078


In [183]:
reg_ols2 = reg(
    df_balanced,
    @formula(ldsal ~ ldnpt + lemp),
    Vcov.cluster(:index)
)


regtable(
    reg_ols2;
    renderSettings = LatexTable(),
    file = joinpath(export_dir, "reg_ols2.tex")
)

┌ Warning: renderSettings is deprecated. Specify render type with render=LatexTable() and file with file=/Users/jorditorresvallverdu/Documents/GitHub/TSE-2025-2026/IO/reg_ols2.tex
└ @ RegressionTables /Users/jorditorresvallverdu/.julia/packages/RegressionTables/wzLBu/src/regtable.jl:441


\begin{tabular}{lr}
\toprule
            & \multicolumn{1}{c}{ldsal} \\ 
\midrule
(Intercept) &                  2.723*** \\ 
            &                   (0.125) \\ 
ldnpt       &                  0.533*** \\ 
            &                   (0.036) \\ 
lemp        &                  0.429*** \\ 
            &                   (0.048) \\ 
\midrule
$N$         &                       856 \\ 
$R^2$       &                     0.913 \\ 
\bottomrule
\end{tabular}


In [184]:
#first differences

using ShiftedArrays

    df3 = deepcopy(main)

    sort!(df3, [:index, :yr])

    df3 = transform(
        groupby(df, :index),
        :ldsal => (x -> x .- lag(x, 1) ) => :dif_ldsal,
        :ldnpt => (x -> x .- lag(x, 1)) => :dif_ldnpt,
        :lemp => (x -> x .- lag(x, 1)) => :dif_lemp,
    )

    first_dif= dropmissing(df3, [:dif_ldsal, :dif_ldnpt, :dif_lemp])

    reg_ols_fd = reg(
    first_dif,
    @formula(dif_ldsal ~ dif_ldnpt + dif_lemp),
    Vcov.cluster(:index)
 )

 regtable(
    reg_ols_fd;
    renderSettings = LatexTable(),
    file = joinpath(export_dir, "reg_ols_fd.tex")
)


┌ Warning: renderSettings is deprecated. Specify render type with render=LatexTable() and file with file=/Users/jorditorresvallverdu/Documents/GitHub/TSE-2025-2026/IO/reg_ols_fd.tex
└ @ RegressionTables /Users/jorditorresvallverdu/.julia/packages/RegressionTables/wzLBu/src/regtable.jl:441


\begin{tabular}{lr}
\toprule
            & \multicolumn{1}{c}{dif\_ldsal} \\ 
\midrule
(Intercept) &                       0.170*** \\ 
            &                        (0.013) \\ 
dif\_ldnpt  &                          0.054 \\ 
            &                        (0.029) \\ 
dif\_lemp   &                       0.870*** \\ 
            &                        (0.047) \\ 
\midrule
$N$         &                          1,571 \\ 
$R^2$       &                          0.530 \\ 
\bottomrule
\end{tabular}


In [185]:
#within group

reg_fe = reg(
    df,
    @formula(ldsal ~ ldnpt + lemp + fe(index)), 
    Vcov.cluster(:index)
)


 regtable(
    reg_fe;
    renderSettings = LatexTable(),
    file = joinpath(export_dir, "reg_fe.tex")
)


┌ Warning: renderSettings is deprecated. Specify render type with render=LatexTable() and file with file=/Users/jorditorresvallverdu/Documents/GitHub/TSE-2025-2026/IO/reg_fe.tex
└ @ RegressionTables /Users/jorditorresvallverdu/.julia/packages/RegressionTables/wzLBu/src/regtable.jl:441


\begin{tabular}{lr}
\toprule
                    & \multicolumn{1}{c}{ldsal} \\ 
\midrule
ldnpt               &                  0.311*** \\ 
                    &                   (0.037) \\ 
lemp                &                  0.753*** \\ 
                    &                   (0.061) \\ 
\midrule
index Fixed Effects &                       Yes \\ 
\midrule
$N$                 &                     2,440 \\ 
$R^2$               &                     0.976 \\ 
Within-$R^2$        &                     0.536 \\ 
\bottomrule
\end{tabular}


In [186]:
using DataFrames, GLM, StatsBase, ShiftedArrays, StatsModels, CategoricalArrays

dfS = deepcopy(df)              
sort!(dfS, [:index, :yr])

# next period year within firm
dfS = transform(groupby(dfS, :index),
    :yr => (x -> lead(x, 1)) => :yr_next
)

# survival: observed next period if yr_next exists (and optionally if it's exactly +5)
dfS.survive = .!ismissing.(dfS.yr_next)

# optional strict version (recommended): require consecutive 5-year step
dfS.survive_strict = (.!ismissing.(dfS.yr_next)) .& ((dfS.yr_next .- dfS.yr) .== 5)



probit_op = glm(
    @formula(survive_strict ~ ldinv + ldnpt + yr + sic3),
    dfS,
    Binomial(),
    ProbitLink(),
    contrasts = Dict(
        :yr   => DummyCoding(),
        :sic3 => DummyCoding()
    )
)

 regtable(
    probit_op;
    renderSettings = LatexTable(),
    file = joinpath(export_dir, "probit_op.tex")
)


coeftable(probit_op)

dfS.Phat = predict(probit_op)




┌ Warning: renderSettings is deprecated. Specify render type with render=LatexTable() and file with file=/Users/jorditorresvallverdu/Documents/GitHub/TSE-2025-2026/IO/probit_op.tex
└ @ RegressionTables /Users/jorditorresvallverdu/.julia/packages/RegressionTables/wzLBu/src/regtable.jl:441


2971-element Vector{Float64}:
 0.69044158247321
 0.5537398194190638
 0.4013296503104493
 0.6531405328889203
 0.6543525725104897
 4.601852576336359e-15
 0.21256519508453287
 0.2209902813587134
 0.8561835576071173
 0.7586512344274903
 ⋮
 0.39814535186946765
 0.3383268066242626
 0.6525549165858056
 0.5027973001414376
 5.98260466647034e-18
 0.5819675916263163
 0.5079426472367088
 9.067606160901689e-17
 3.845970794556528e-16

----

**Olley Pakes** (1996)



In [187]:
##Olly/Pakes 

#step 1
Kit = df.ldnpt
Iit = df.ldinv
Lit = df.lemp
Yit = df.ldsal

function poly2(k, i)
    n = length(k)
    hcat(
        ones(n),
        k, i,
        k.^2, i.^2,
        k .* i,
        k.^3, i.^3,
        k.^2 .* i,
        i.^2 .* k
    )
end

Z = poly2(Kit, Iit)          # this is φ(i,k) basis
X1 = hcat(Lit, Z)            # include labor linearly

β1 = (X1'X1) \ (X1'Yit)

beta_l_hat = β1[1]          
phi_hat    = Z * β1[2:end]   

df.ϕ_t_hat = phi_hat



2971-element Vector{Union{Missing, Float64}}:
 6.2097814308356
 6.250502801426724
 5.810396907007858
 4.095766380549088
 4.575993155775011
 4.575506937900109
 3.7173814404062773
 4.201416215663969
 4.415133564488671
 4.519976453972892
 ⋮
 3.7435321925499268
 4.006077586046412
 4.599241630536814
 4.805087902977735
 5.189322623894464
 5.5248732585964
 5.636494889143368
 5.603722584650479
 3.9390602283397045

In [188]:
using DataFrames, ShiftedArrays

sort!(df, [:index, :yr])

df2 = transform(
    groupby(df, :index),
    :yr => (x -> lead(x, 1)) => :lead_yr
)

df2.survived= .!ismissing.(df2.lead_yr)

#drop last year of the panel 

data_step2= filter(row->row.yr<88, df2)


Row,index,sic3,yr,ldsal,lemp,ldnpt,ldrst,ldrnd,ldinv,ϕ_t_hat,lead_yr,survived
,Int16?,Int16?,Int8?,Float32,Float32,Float32,Float32?,Float32?,Float32?,Float64?,Int8?,Bool
1,1,374,73,7.01896,2.51098,7.04537,3.7509,2.03022,4.98944,6.20978,78,true
2,1,374,78,7.09394,2.67104,7.18584,3.79655,2.04228,4.90343,6.2505,83,true
3,1,374,83,6.4082,2.00647,7.4539,3.92492,2.0334,3.82753,5.8104,missing,false
4,2,366,78,3.52628,-0.0523465,2.4056,2.61583,1.12504,0.633175,4.09577,83,true
5,2,366,83,4.37366,0.640801,3.4784,3.22696,1.77073,1.91537,4.57599,88,true
6,3,358,78,3.80885,-0.543005,1.72808,0.746074,-1.05687,-0.748376,3.71738,83,true
7,3,358,83,3.85115,-0.478036,2.69638,1.32711,0.265216,0.725182,4.20142,missing,false
8,4,381,73,4.25076,0.210261,3.03931,2.75336,0.812571,2.16101,4.41513,78,true
9,4,381,78,4.45412,0.330023,3.32176,2.42518,0.366463,2.04567,4.51998,83,true


In [189]:
using DataFrames, ShiftedArrays, GLM

sort!(df, [:index, :yr])

df2 = transform(
    groupby(df, :index),
    :yr => (x -> lead(x, 1)) => :lead_yr
)

df2.survived = .!ismissing.(df2.lead_yr)

data_step2 = filter(row -> row.yr < 88, df2)

function poly3(k, i)
    n = length(k)
    hcat(
        ones(n),
        k, i,
        k.^2, i.^2,
        k .* i,
        k.^3, i.^3,
        k.^2 .* i,
        i.^2 .* k
    )
end

df_clean = dropmissing(data_step2, [:survived, :ldinv, :ldnpt])

Iit = df_clean.ldinv
Kit = df_clean.ldnpt
y   = df_clean.survived

X = poly3(Kit, Iit)

probit = glm(X, y, Binomial(), ProbitLink())

df_clean.P̂ = predict(probit)

df_clean_small = select(df_clean, :yr, :index, :P̂)


Row,yr,index,P̂
,Int8?,Int16?,Float64
1,73,1,0.813965
2,78,1,0.794649
3,83,1,0.510251
4,78,2,0.58963
5,83,2,0.651903
6,78,3,0.549118
7,83,3,0.587008
8,73,4,0.68706
9,78,4,0.667888


In [190]:
#step 3. 

df_clean


df_total = innerjoin(
    df,
    select(df_clean, :yr, :index, :P̂),
    on = [:index, :yr]
)

#we generate t+1 leads. 
df_total_final= transform(groupby(df_total, :index),
    :ldsal => (x->lead(x)) => :ldsal_lead,
    :lemp => (x->lead(x)) => :lemp_lead,
    :ldnpt => (x->lead(x)) => :ldnpt_lead,
    :ldrnd => (x->lead(x)) => :ldrnd_lead,
    :ldnpt => (x->lag(x)) => :ldnpt_lag,
    :lemp =>(x->lag(x)) => :lemp_lag
)





Row,index,sic3,yr,ldsal,lemp,ldnpt,ldrst,ldrnd,ldinv,ϕ_t_hat,P̂,ldsal_lead,lemp_lead,ldnpt_lead,ldrnd_lead,ldnpt_lag,lemp_lag
,Int16?,Int16?,Int8?,Float32,Float32,Float32,Float32?,Float32?,Float32?,Float64?,Float64,Float32?,Float32?,Float32?,Float32?,Float32?,Float32?
1,1,374,73,7.01896,2.51098,7.04537,3.7509,2.03022,4.98944,6.20978,0.813965,7.09394,2.67104,7.18584,2.04228,missing,missing
2,1,374,78,7.09394,2.67104,7.18584,3.79655,2.04228,4.90343,6.2505,0.794649,6.4082,2.00647,7.4539,2.0334,7.04537,2.51098
3,1,374,83,6.4082,2.00647,7.4539,3.92492,2.0334,3.82753,5.8104,0.510251,missing,missing,missing,missing,7.18584,2.67104
4,2,366,78,3.52628,-0.0523465,2.4056,2.61583,1.12504,0.633175,4.09577,0.58963,4.37366,0.640801,3.4784,1.77073,missing,missing
5,2,366,83,4.37366,0.640801,3.4784,3.22696,1.77073,1.91537,4.57599,0.651903,missing,missing,missing,missing,2.4056,-0.0523465
6,3,358,78,3.80885,-0.543005,1.72808,0.746074,-1.05687,-0.748376,3.71738,0.549118,3.85115,-0.478036,2.69638,0.265216,missing,missing
7,3,358,83,3.85115,-0.478036,2.69638,1.32711,0.265216,0.725182,4.20142,0.587008,missing,missing,missing,missing,1.72808,-0.543005
8,4,381,73,4.25076,0.210261,3.03931,2.75336,0.812571,2.16101,4.41513,0.68706,4.45412,0.330023,3.32176,0.366463,missing,missing
9,4,381,78,4.45412,0.330023,3.32176,2.42518,0.366463,2.04567,4.51998,0.667888,4.19622,0.276115,3.16302,-0.111755,3.03931,0.210261


In [191]:

#filter for no missing in these key variables

df_total_final=dropmissing(df_total_final, [:ldsal_lead, :ldnpt_lead, :lemp_lead, :P̂, :ϕ_t_hat , :ldnpt, :ldrnd, :ldrnd_lead])

#define main variables 
y_tlead= df_total_final.ldsal_lead
k_tlead=df_total_final.ldnpt_lead
l_tlead= df_total_final.lemp_lead
k_t=df_total_final.ldnpt

p_t=df_total_final.P̂
phi_t= df_total_final.ϕ_t_hat

#dependent variable
y= @. y_tlead- beta_l_hat*l_tlead


#polynomial on p_t, phi_t

function poly_final(ωt, p_t)

    n = length(p_t)

    X = hcat(
        ones(n),
        p_t, ωt, 
        p_t.^2, ωt.^2,
        p_t .* ωt, 
        p_t.^3, ωt.^3, 
        p_t.^2 .* ωt, 
        ωt.^2 .* p_t, 
    )
    return X
    

end 


beta_l_hat

0.5246170338398102

In [192]:
function nonlinear_solv(β_k, p_t, y)

            ωt= phi_t .- β_k .* k_t
            y_reg= y .- β_k .* k_tlead
            X= poly_final(ωt, p_t)

            beta_try= (X'X)\(X'y_reg)

            ϵ= y_reg - X*beta_try

            sse = sum(ϵ.^2)
            return sse

end 



nonlinear_solv (generic function with 1 method)

In [193]:
   using Optim, PrettyTables
    θ0= [0.4]

   ob1(βk) = nonlinear_solv(βk, p_t, y)

   res1 = optimize(ob1, 0.01, 1.0, Brent())

   β_k = Optim.minimizer(res1)

   final_result_op1= [β_k, beta_l_hat]


op = DataFrame(
    Variable = ["β_k", "β_l"],
    Coefficient   = final_result_op1
)

#this is to put into latex in a correct manner.
show(stdout, MIME("text/latex"), op)


#note: standard errors need to be computed by bootstrap!


\begin{tabular}{r|cc}
	& Variable & Coefficient\\
	\hline
	& String & Float64\\
	\hline
	1 & β\_k & 0.438765 \\
	2 & β\_l & 0.524617 \\
\end{tabular}


14

In [194]:
   #estimating productivity
   
   df_total_final.ω_t_noRD= phi_t .- β_k .* k_t 


1079-element Vector{Float64}:
 3.1185153366937985
 3.097603323492671
 3.040272375063496
 2.9591584569612692
 3.0815895968780396
 3.062505003179255
 3.0855979244903855
 3.0958932452537664
 3.0204790718178094
 3.070016096352148
 ⋮
 2.9976995082840245
 2.9701174272639537
 2.9497369011098744
 3.044705916222755
 2.908715548221387
 3.1630325127601857
 2.9555187918096646
 3.0585459575663094
 2.993248758638638



---

In [195]:
##adding rd_investment 


Kit = df.ldnpt
Iit = df.ldinv
RDt = df.ldrnd
Lit = df.lemp
Yit = df.ldsal

function phi_basis(k, i, rd)
    n = length(k)
    hcat(
        ones(n),
        k, i, rd,
        k.^2, i.^2, rd.^2,
        k .* i, k .* rd, i .* rd,
        k.^3, i.^3, rd.^3,
        k.^2 .* i, k.^2 .* rd,
        i.^2 .* k, i.^2 .* rd,
        rd.^2 .* k, rd.^2 .* i
    )
end

Z = phi_basis(Kit, Iit, RDt)      # φ(i,k,R) basis
X1 = hcat(Lit, Z)                # labor enters linearly
β1 = (X1'X1) \ (X1'Yit)

beta_l_hat = β1[1]               
phi_hat    = Z * β1[2:end]       
df.ϕ_t_hat = phi_hat


2971-element Vector{Union{Missing, Float64}}:
 6.243001884312949
 6.26363810794664
 5.767861892376865
 4.128996421926241
 4.57419230678595
 4.655861366039607
 3.6908438557637675
 4.174429150374104
 4.33854467144828
 4.469421853561073
 ⋮
 3.719645270664543
 4.005763742090393
 4.618043569609353
 4.807641852537701
 5.162829674925504
 5.580860372615991
 5.711735850514589
 5.715004626045854
 4.223291449694387

In [196]:

#filter for no missing in these key variables, again; should do this more abstractly and efficently. 

#left join
select!(df_total_final, Not(:ϕ_t_hat))

df_total_final = leftjoin(
    df_total_final,
    select(df, :index, :yr, :ϕ_t_hat),
    on = [:index, :yr]
)



#define main variables 
y_tlead= df_total_final.ldsal_lead
k_tlead=df_total_final.ldnpt_lead
l_tlead= df_total_final.lemp_lead
rd_tlead= df_total_final.ldrnd_lead

k_t=df_total_final.ldnpt
rd_t=df_total_final.ldrnd

p_t=df_total_final.P̂
phi_t = df_total_final.ϕ_t_hat

#dependent variable
y= @. y_tlead- beta_l_hat*l_tlead


#polynomial on p_t, phi_t

function poly_final(ωt, p_t)

    n = length(p_t)

    X = hcat(
        ones(n),
        p_t, ωt, 
        p_t.^2, ωt.^2,
        p_t .* ωt, 
        p_t.^3, ωt.^3, 
        p_t.^2 .* ωt, 
        ωt.^2 .* p_t, 
    )
    return X
    

end 


beta_l_hat

0.5249235501687262

In [197]:
function nonlinear_solv_2(β, p_t, y)
            β_k, β_rd= β

            ωt= phi_t .- β_k .* k_t - β_rd .* rd_t
            y_reg= y .- β_k .* k_tlead - β_rd.*rd_tlead
            X= poly_final(ωt, p_t)

            beta_try= (X'X)\(X'y_reg)

            ϵ= y_reg - X*beta_try

            sse = sum(ϵ.^2)
            return sse

end 


nonlinear_solv_2 (generic function with 1 method)

In [198]:
   using Optim
    θ0= [0.7, 0.9]

   ob2(βk) = nonlinear_solv_2(βk, p_t, y)

   res2 = optimize(ob2, θ0, NelderMead())

   β= Optim.minimizer(res2)

   β_k_hat= β[1]
   β_rd_hat=β[2]


op_2 = DataFrame(
    Variable = ["β_k", "β_i", "β_l"],
    Coefficient   = [β_k_hat, β_rd_hat, beta_l_hat]
)

show(stdout, MIME("text/latex"), op_2)





\begin{tabular}{r|cc}
	& Variable & Coefficient\\
	\hline
	& String & Float64\\
	\hline
	1 & β\_k & 0.380463 \\
	2 & β\_i & 0.0229861 \\
	3 & β\_l & 0.524924 \\
\end{tabular}


14

---

**Question 2.6** 

In [199]:
#for the first case. 

df_total_final.ω_t_hat_2= ωt= phi_t .- β_k_hat .* k_t - β_rd_hat .* rd_t

df_total_final2= transform(groupby(df_total_final, :yr),
:ω_t_hat_2 => mean => :ω_t_hat_average,
:ldsal => (x-> exp.(x)./sum(exp.(x))) => :sjt,
:ldsal => (x -> length(x)) => :Nt
)

sjt= df_total_final2.sjt
Nt= df_total_final2.Nt
ω_t_hat_average= df_total_final2.ω_t_hat_average
ω_t_hat_j=df_total_final2.ω_t_hat_2

df_total_final2.covariance_op= (sjt .- (1.0 ./Nt)) .*(ω_t_hat_j .-ω_t_hat_average)


df_total_final2= transform(groupby(df_total_final2, :yr), 
:covariance_op => sum => :covariance_op_average,

)

results= [df_total_final2.ω_t_hat_average, df_total_final2.covariance_op_average]


2-element Vector{Vector{Float64}}:
 [3.292377802404195, 3.2818783334332244, 3.2818783334332244, 3.2818783334332244, 3.292377802404195, 3.2818783334332244, 3.292377802404195, 3.2818783334332244, 3.292377802404195, 3.2818783334332244  …  3.2818783334332244, 3.2818783334332244, 3.292377802404195, 3.2818783334332244, 3.2818783334332244, 3.292377802404195, 3.2818783334332244, 3.2818783334332244, 3.292377802404195, 3.2818783334332244]
 [0.24348183544295227, 0.26694109449347453, 0.26694109449347453, 0.26694109449347453, 0.24348183544295227, 0.26694109449347453, 0.24348183544295227, 0.26694109449347453, 0.24348183544295227, 0.26694109449347453  …  0.26694109449347453, 0.26694109449347453, 0.24348183544295227, 0.26694109449347453, 0.26694109449347453, 0.24348183544295227, 0.26694109449347453, 0.26694109449347453, 0.24348183544295227, 0.26694109449347453]

In [200]:
##for the second case. 

df_total_final2= filter(row->row.sic3==375, df_total_final)


Row,index,sic3,yr,ldsal,lemp,ldnpt,ldrst,ldrnd,ldinv,P̂,ldsal_lead,lemp_lead,ldnpt_lead,ldrnd_lead,ldnpt_lag,lemp_lag,ω_t_noRD,ϕ_t_hat,ω_t_hat_2
,Int16?,Int16?,Int8?,Float32,Float32,Float32,Float32?,Float32,Float32?,Float64,Float32,Float32,Float32,Float32,Float32?,Float32?,Float64,Float64?,Float64
1,626,375,73,5.50471,0.938835,3.32937,0.943609,-0.4809,1.75769,0.643522,5.59164,1.03496,3.13264,0.388936,missing,missing,3.0503,4.48866,3.23302
2,626,375,78,5.59164,1.03496,3.13264,1.35218,0.388936,2.09168,0.677755,5.4656,1.07432,3.96635,0.804787,3.32937,0.938835,3.07416,4.38243,3.18164


In [201]:
df_total_final

Row,index,sic3,yr,ldsal,lemp,ldnpt,ldrst,ldrnd,ldinv,P̂,ldsal_lead,lemp_lead,ldnpt_lead,ldrnd_lead,ldnpt_lag,lemp_lag,ω_t_noRD,ϕ_t_hat,ω_t_hat_2
,Int16?,Int16?,Int8?,Float32,Float32,Float32,Float32?,Float32,Float32?,Float64,Float32,Float32,Float32,Float32,Float32?,Float32?,Float64,Float64?,Float64
1,1,374,73,7.01896,2.51098,7.04537,3.7509,2.03022,4.98944,0.813965,7.09394,2.67104,7.18584,2.04228,missing,missing,3.11852,6.243,3.51583
2,1,374,78,7.09394,2.67104,7.18584,3.79655,2.04228,4.90343,0.794649,6.4082,2.00647,7.4539,2.0334,7.04537,2.51098,3.0976,6.26364,3.48275
3,2,366,78,3.52628,-0.0523465,2.4056,2.61583,1.12504,0.633175,0.58963,4.37366,0.640801,3.4784,1.77073,missing,missing,3.04027,4.129,3.1879
4,3,358,78,3.80885,-0.543005,1.72808,0.746074,-1.05687,-0.748376,0.549118,3.85115,-0.478036,2.69638,0.265216,missing,missing,2.95916,3.69084,3.05767
5,4,381,73,4.25076,0.210261,3.03931,2.75336,0.812571,2.16101,0.68706,4.45412,0.330023,3.32176,0.366463,missing,missing,3.08159,4.33854,3.16352
6,4,381,78,4.45412,0.330023,3.32176,2.42518,0.366463,2.04567,0.667888,4.19622,0.276115,3.16302,-0.111755,3.03931,0.210261,3.06251,4.46942,3.19719
7,7,394,73,7.54862,3.50255,6.35166,5.47065,3.94797,4.8184,0.824826,7.52507,3.35197,6.3532,3.48677,missing,missing,3.0856,5.8421,3.33478
8,7,394,78,7.52507,3.35197,6.3532,5.32021,3.48677,4.58983,0.803268,6.91924,2.93055,6.4097,3.43927,6.35166,3.50255,3.09589,5.85786,3.36056
9,8,355,73,7.30756,3.157,5.67673,4.4641,2.4318,4.5587,0.820489,7.04852,2.91777,5.61488,2.59354,missing,missing,3.02048,5.50646,3.29078


In [202]:
##Question 2.7

#ols
df.ω_t_ols= df.ldsal .- reg_ols1.coef[1] .- reg_ols1.coef[2] .* df.ldnpt .- reg_ols1.coef[3] .* df.lemp

#FE
df.ω_t_fe = df.ldsal  .- reg_fe.coef[1].* df.ldnpt  .- reg_fe.coef[2].* df.lemp



2971-element Vector{Float64}:
 2.9365356818591604
 2.847342575695577
 2.5783329693928234
 2.81700241320292
 2.8088469129493534
 3.727991116807022
 3.679699925799152
 3.3717522161465916
 3.146626078285593
 3.171946770403968
 ⋮
 3.0873424894168746
 3.017885257000613
 3.8173224678939426
 3.3440766361944156
 3.8361915779410096
 2.7398060107988447
 2.698603419586906
 3.388253404290575
 4.430773234760476

In [203]:
df_total_final

Row,index,sic3,yr,ldsal,lemp,ldnpt,ldrst,ldrnd,ldinv,P̂,ldsal_lead,lemp_lead,ldnpt_lead,ldrnd_lead,ldnpt_lag,lemp_lag,ω_t_noRD,ϕ_t_hat,ω_t_hat_2
,Int16?,Int16?,Int8?,Float32,Float32,Float32,Float32?,Float32,Float32?,Float64,Float32,Float32,Float32,Float32,Float32?,Float32?,Float64,Float64?,Float64
1,1,374,73,7.01896,2.51098,7.04537,3.7509,2.03022,4.98944,0.813965,7.09394,2.67104,7.18584,2.04228,missing,missing,3.11852,6.243,3.51583
2,1,374,78,7.09394,2.67104,7.18584,3.79655,2.04228,4.90343,0.794649,6.4082,2.00647,7.4539,2.0334,7.04537,2.51098,3.0976,6.26364,3.48275
3,2,366,78,3.52628,-0.0523465,2.4056,2.61583,1.12504,0.633175,0.58963,4.37366,0.640801,3.4784,1.77073,missing,missing,3.04027,4.129,3.1879
4,3,358,78,3.80885,-0.543005,1.72808,0.746074,-1.05687,-0.748376,0.549118,3.85115,-0.478036,2.69638,0.265216,missing,missing,2.95916,3.69084,3.05767
5,4,381,73,4.25076,0.210261,3.03931,2.75336,0.812571,2.16101,0.68706,4.45412,0.330023,3.32176,0.366463,missing,missing,3.08159,4.33854,3.16352
6,4,381,78,4.45412,0.330023,3.32176,2.42518,0.366463,2.04567,0.667888,4.19622,0.276115,3.16302,-0.111755,3.03931,0.210261,3.06251,4.46942,3.19719
7,7,394,73,7.54862,3.50255,6.35166,5.47065,3.94797,4.8184,0.824826,7.52507,3.35197,6.3532,3.48677,missing,missing,3.0856,5.8421,3.33478
8,7,394,78,7.52507,3.35197,6.3532,5.32021,3.48677,4.58983,0.803268,6.91924,2.93055,6.4097,3.43927,6.35166,3.50255,3.09589,5.85786,3.36056
9,8,355,73,7.30756,3.157,5.67673,4.4641,2.4318,4.5587,0.820489,7.04852,2.91777,5.61488,2.59354,missing,missing,3.02048,5.50646,3.29078


In [204]:
#plot trends 

df_allmeasures = innerjoin(
    df,
    select(df_total_final, :yr, :index, :ω_t_noRD , :ω_t_hat_2),
    on = [:index, :yr]
)


 g = groupby(df_allmeasures, :yr)

out= combine(g, 

            :ω_t_noRD=> mean => :ω_t_1 ,
            :ω_t_hat_2=> mean => :ω_t_2 ,
            :ω_t_ols => mean => :ω_t_ols ,
            :ω_t_fe => mean => :ω_t_fe
        )



##nice results!
show(stdout, MIME("text/latex"), out)




\begin{tabular}{r|ccccc}
	& yr & ω\_t\_1 & ω\_t\_2 & ω\_t\_ols & ω\_t\_fe\\
	\hline
	& Int8? & Float64 & Float64 & Float64 & Float64\\
	\hline
	1 & 73 & 3.0564 & 3.29238 & -0.106186 & 3.16725 \\
	2 & 78 & 3.06012 & 3.28188 & -0.0599187 & 3.23616 \\
\end{tabular}


14

---

**ACF**

**Question 2.8**

The issue is that then 

$$

 y_{it}  + p_{it}= \beta_{0} + \beta_{l}l_{it} + \beta_{l}l_{it} + \omega_{it} + \epsilon_{it}

$$

$$

 y_{it} = \beta_{0} + \beta_{l}l_{it} + \beta_{l}l_{it} + \omega_{it} -p_{it} + \epsilon_{it}

$$

The error also contains $p_{it}$ which is endogeneous and can determine labor or capital , conditional on productivity, choices. For example, firms, independent of productivity, may have prestige, market power over a region that leads to higher prices than other firms. This is not solved by OP.  

Solution: either we defend that $$ (l_{it}, k_{it}|\omega_{it}) \perp p_{it}$$ -i.e homogeneous goods industry, small markups etc- or we find an instrument that affects prices and is exogeneous to input choices (i.e demand side, for example) - other product characteristics (á la BLP)

**Question 2.9**
Now we will estimate the production function using the methodology developed by ACF. The idea is the following: 


We have the following production function:

$$y_{it}= \beta_{0} + \beta_{l} l_{it} + \beta_{k} k_{it} + \omega_{it} + \epsilon_{it} $$

Under ACF we make the following assumptions:

1. **Information set**: the firm information set at period $t$, denoted $I_{t}$, includes current and past productivity shocks $\{\omega_{\tau}\}_{\tau=0}^{t}$ but not future shocks $\{\omega_{\tau}\}_{\tau=t+1}^{\infty}$. Transitory shocks satisfy $ \mathbf{E}[\epsilon_{it}|I_{t-1}]=0$.
2. **Markov process** productivity shocks evolve according to: $$ P(\omega_{it+1}|I_{t})= P(\omega_{it+1}|\omega_{it})$$.
3. **Timing decision** Firms accumulate capital according to the process $$k_{it}=k(k_{it-1}, i_{it-1}) $$. That is, labor is the only non-dynamic input and has flexibility, while capital and investment depend on the previous period. 
4. **Scalar Unobservable** Firm's investment decisions are given by $i_{it}=f_{t}(k_{it},\omega_{it})$
5. **Invertibility** $f_{t}$ is strictly increasing in $\omega_{it}$, which implies that the function is injective and thus we can invert it: $\omega_{it}= f_{t}^{-1}(k_{t}, i_{t})$

---

Assume we observe $l_{it}, k_{it}, y_{it}, i_{it}$, amd we have T fixed and $N \rightarrow \infty$ . Then, ACF proposes to estimate the following model: 

$$ 
y_{it}= \beta_{0} + \beta_{l} l_{it} + \beta_{k} k_{it}+ f_{t}^{-1}(k_{t}, i_{t}) + \epsilon_{it}
$$

$$ 
y_{it}= \beta_{l} l_{it} + \Phi(k_{it},l_{it}, i_{it}) + \epsilon_{it}
$$

And estimate it using the condition $ \mathbf{E}[\epsilon_{it}|I_{t-1}]=0$, thus:

$$
\mathbf{E}[y_{it}-\Phi(k_{it},l_{it}, i_{it})|I_{t-1}]=0
$$
To estimate  $\hat{\Phi}$. Then in a second step we use the fact that 

$$

\mathbf{E}[\epsilon_{it+1}+ \xi_{it+1}|I_{t}]=0

$$

$$

\mathbf{E}[y_{jt+1}-\beta_{0}-\beta_{l}l_{t+1}- \beta_{k}k_{t+1} - g(\hat{P}, \hat{\Phi_{j,t}}-\beta_{0}-\beta_{l}l_{t}- \beta_{k}k_{t})|I_{t}]=0

$$

Where $I_{j,t}=1, k_{jt+1}, l_{jt}, \Phi_{jt}, l_{jt+1}$

So that the moments are: 

$$

(y_{jt+1}-\beta_{0}-\beta_{l}l_{t+1}- \beta_{k}k_{t+1} - g(\hat{P}, \hat{\Phi_{j,t}}-\beta_{0}-\beta_{l}l_{t}- \beta_{k}k_{t})) \otimes (I_{j,t} )=0

$$

We can solve this by GMM.



In [205]:
#Step 1. 

#define variables, maybe redundant, but needed. 

df_acf1 = dropmissing(df, [:ldsal, :ldnpt, :lemp, :ldinv])
df_acf1 = innerjoin(
    df_acf1,
    select(df_clean, :yr, :index, :P̂),
    on = [:index, :yr]
)

Kit = df_acf1.ldnpt
Iit = df_acf1.ldinv
Lit = df_acf1.lemp
Yit = df_acf1.ldsal


#Flexible function that I can use with choosen number of polynomials K.
function poly4(k, i, l)
    n = length(k)
    X = hcat(
        ones(n),

        k, i, l,
        k.^2, i.^2, l.^2,
        k .* i, k .* l, i .* l,
        k.^3, i.^3, l.^3,
        k.^2 .* i, k.^2 .* l,
        i.^2 .* k, i.^2 .* l,
        l.^2 .* k, l.^2 .* i
    )
    return X
end


X= poly4(Kit, Iit, Lit)

Beta_acf_1= (X'X)\(X'Yit)

df_acf1.phi_acf= X*Beta_acf_1

2285-element Vector{Float64}:
 7.53119157652836
 7.641139699560529
 6.822437470425208
 4.0204859619334075
 4.855221475900268
 3.3671283560729637
 3.897064034385158
 4.360103081629631
 4.6064228277593795
 4.480564227518233
 ⋮
 9.014018650997805
 9.025359010584127
 4.838020151504794
 3.2444264209391993
 3.464830448870139
 4.182577653188788
 4.650925969899328
 7.250070642237443
 7.537362707766947

In [206]:

df_acf2 = deepcopy(df_acf1)
sort!(df_acf2, [:index, :yr])

df_acf2 = transform(groupby(df_acf2, :index),
    :ldsal => (x -> lead(x)) => :ldsal_lead,
    :lemp  => (x -> lead(x)) => :lemp_lead,
    :ldnpt => (x -> lead(x)) => :ldnpt_lead,
    :ldnpt => (x -> lag(x))  => :ldnpt_lag,
    :lemp => (x -> lag(x)) => :lemp_lag
)


df_acf = dropmissing(df_acf2,
    [:phi_acf, :ldsal_lead, :lemp_lead, :ldnpt_lead]
)


#variables 
y_tlead= df_acf.ldsal_lead
k_tlead=df_acf.ldnpt_lead
l_tlead= df_acf.lemp_lead
k_tlag=df_acf.ldnpt_lag
l_tlag=df_acf.lemp_lag
l_t = df_acf.lemp
k_t = df_acf.ldnpt
i_t= df_acf.ldinv


p_t=df_acf.P̂
phi_t= df_acf.phi_acf


#two-step GMM, step 1.

function moments(θβ::AbstractVector, Z::AbstractMatrix, n)

    β0, βl, βk = θβ

    ωt = phi_t .- β0 .- βl .* l_t .- βk .* k_t

    # g(ω,p) basis
    X = poly_final(ωt, p_t)

    # left-hand side for g-regression
    ytilde = y_tlead .- β0 .- βl .* l_tlead .- βk .* k_tlead

    # "profile" γ: OLS each iteration (stable)
    γ = (X'X) \ (X'ytilde)
    ghat = X * γ

    # innovation / residual
    ν = ytilde .- ghat

    # moments
    g = (Z' * ν) / n
    return g, ν, γ
end



function gmm_objective(θ, Z, n, W)
    g, _, _ = moments(θ, Z, n)
    return n * (g' * W * g)
end


function s_robust(θ, Z, n)
    _, ν, γ = moments(θ, Z, n)
    νv = vec(ν)
    M  = Z .* νv
    m=(M' * M) / n
    return m, γ
end


using ForwardDiff, LinearAlgebra

function compute_se(Z, n, θ2, W2, S1)
    gfun(θ) = moments(θ, Z, n)[1]
    D = ForwardDiff.jacobian(gfun, θ2)

    S1 = 0.5 * (S1 + S1')
    W2 = 0.5 * (W2 + W2')

    A = D' * W2 * D
    B = D' * W2 * S1 * W2 * D

    A = 0.5 * (A + A')
    B = 0.5 * (B + B')

    V = inv(A) * B * inv(A) / n
    return sqrt.(diag(V))
end



#full fledge gmm 
function gmm(Z, n, θ0)

    q = size(Z, 2)
    W1 = Matrix(I, q, q)


    #first step
    obj1(θ) = gmm_objective(θ, Z, n, W1)
    res1 = optimize(obj1, θ0, NelderMead())
    θ1 = Optim.minimizer(res1)

    #optimal weighting matrix
    S1, γ = s_robust(θ1, Z, n)
    W2 = pinv(S1)

    #second step

    obj2(θ) = gmm_objective(θ, Z, n, W2)
    res2 = optimize(obj2, θ1, NelderMead())
    θ2 = Optim.minimizer(res2)


    #standard errors

    se= compute_se(Z, n, θ2, W2, S1)



    #return

    param = hcat(
    θ2,
    se)


    return param


end 






gmm (generic function with 1 method)

In [207]:
#define instruments and inputs 

n= length(y_tlead)


#define instruments and inputs 
Z_1= hcat(
ones(length(y_tlead)), 
k_tlead,
l_t, 
k_t
)

#first guess is OLS on this sample
Y0 = df_acf.ldsal
X0 = hcat(ones(n), l_t, k_t)
θ0 = (X0'X0)\(X0'Y0)  # [β0, βl, βk] start

#θ0= vcat(5, 0.2, 0.7) , trying different starting points really does not add

res= gmm(Z_1, n, θ0)

#almost the same results in both cases...


ACF = DataFrame(
    Variable = ["β_0", "β_l", "β_k"],
    Coefficient = res[:, 1],
    StdErr      = res[:, 2]
)


show(stdout, MIME("text/latex"), ACF)



\begin{tabular}{r|ccc}
	& Variable & Coefficient & StdErr\\
	\hline
	& String & Float64 & Float64\\
	\hline
	1 & β\_0 & 3.52753 & 6.87803e10 \\
	2 & β\_l & 0.599619 & 0.133781 \\
	3 & β\_k & 0.392037 & 0.0663812 \\
\end{tabular}


14

In [208]:
#//!\\ pending to create a proper table with latex export.

---

**GMM all together (Wooldridge)** The idea is to account for the estimation of the first stage when doing the inference. Estimates should be the same but standard errors more conservative. 




In [209]:
using ForwardDiff, LinearAlgebra


#variables 
y_tlead= df_acf.ldsal_lead
k_tlead=df_acf.ldnpt_lead
l_tlead= df_acf.lemp_lead
k_tlag=df_acf.ldnpt_lag
l_tlag=df_acf.lemp_lag
l_t = df_acf.lemp
k_t = df_acf.ldnpt
i_t= df_acf.ldinv
Yit=df_acf.ldsal

p_t=df_acf.P̂
phi_t= df_acf.phi_acf


function moments_stacked(θ, Z, Xφ, n)
    Kφ = size(Xφ,2)

    β0, βl, βk = θ[1:3]
    π = θ[4:3+Kφ]

    #block1
    η = Yit - Xφ * π
    g1 = (Xφ' * η) / n

    g1 = (Xφ' * η) / n

    #block2
    φt = Xφ * π
    ωt = φt .- β0 .- βl .* l_t .- βk .* k_t

    Xg = poly_final(ωt, p_t)
    ytilde = y_tlead .- β0 .- βl .* l_tlead .- βk .* k_tlead

    # profile γ
    γ = (Xg'Xg) \ (Xg'ytilde)
    ghat = Xg * γ

    ε = ytilde .- ghat

    g2 = (Z' * ε) / n

    return vcat(g1, g2), η, ε
end



function gmm_objective_full(θ, Z, n, W, Xφ)
    g, _, _ = moments_stacked(θ, Z, Xφ, n)

    return n * (g' * W * g)
end


function s_robust_full(θ, Z, Xφ, n)
    g, η, ε = moments_stacked(θ, Z, Xφ, n)

    M1 = Xφ .* η        
    M2 = Z  .* ε        
    M  = hcat(M1, M2)   

    return (M' * M) / n
end


function compute_se_full(Z, Xφ, n, θ2, W2, S1)
    gfun(θ) = moments_stacked(θ, Z, Xφ, n)[1]

    D = ForwardDiff.jacobian(gfun, θ2)

    S1 = 0.5 * (S1 + S1')
    W2 = 0.5 * (W2 + W2')

    A = D' * W2 * D
    B = D' * W2 * S1 * W2 * D

    A = 0.5 * (A + A')
    B = 0.5 * (B + B')

    #for safety
    V = (A \ B) / A'
    V = 0.5 * (V + V') / n

    d = diag(V)
    d = map(x -> x < 0 ? NaN : x, d)
    return sqrt.(d)
end




compute_se_full (generic function with 1 method)

In [210]:
function gmm_full(Z, n, θ0, Xφ)

    Kφ = size(Xφ,2)
    q2 = size(Z,2)
    q  = Kφ + q2
    W1 = Matrix(I, q, q)

    # step 1
    obj1(θ) = gmm_objective_full(θ, Z, n, W1, Xφ)
    res1 = optimize(obj1, θ0, NelderMead())
    θ1 = Optim.minimizer(res1)

    # robust S and ridge-regularized W
    S1 = s_robust_full(θ1, Z, Xφ, n)
    S1 = 0.5*(S1 + S1')
    λ  = 1e-8 * tr(S1) / size(S1,1)
    S1r = S1 + λ*I
    W2  = inv(S1r)

    # step 2
    obj2(θ) = gmm_objective_full(θ, Z, n, W2, Xφ)
    res2 = optimize(obj2, θ1, NelderMead())
    θ2 = Optim.minimizer(res2)

    # SEs (use same S that defines W2)
    se = compute_se_full(Z, Xφ, n, θ2, W2, S1r)

    return hcat(θ2, se)
end



gmm_full (generic function with 1 method)

In [211]:
function poly4_noconst(k, i, l)
    n = length(k)
    X = hcat(
        k, i, l,
        k.^2, i.^2, l.^2,
        k .* i, k .* l, i .* l,
        k.^3, i.^3, l.^3,
        k.^2 .* i, k.^2 .* l,
        i.^2 .* k, i.^2 .* l,
        l.^2 .* k, l.^2 .* i
    )
    return X
end

poly4_noconst (generic function with 1 method)

In [212]:
Xφ = poly4(k_t, i_t, l_t)
Kφ = size(Xφ, 2)


n= length(y_tlead)

#define instruments and inputs 

Z_1 = hcat(ones(n), k_tlead, l_t, k_t)
  # maybe add Kit too; here I need more instruments because I have a lot of parameters to estimate...or maybe they are colinear?


#first guess is OLS on this sample
X= hcat(
    ones(length(y_tlead)), 
    l_t, 
    k_t
)
Y0 = df_acf.ldsal

θ0= (X'X)\(X'Y0)

θ0_1=vcat(θ0, Beta_acf_1)

#
param= gmm_full(Z_1, n, θ0_1, Xφ)
    #define theta0




ACF_woold = DataFrame(
    Variable = ["β_0", "β_l", "β_k"],
    Coefficient = [param[1, 1], param[2,1], param[3,1]], 
    StdErr      = [param[1, 2], param[2, 2], param[3,2]]
)


show(stdout, MIME("text/latex"), ACF_woold)


\begin{tabular}{r|ccc}
	& Variable & Coefficient & StdErr\\
	\hline
	& String & Float64 & Float64\\
	\hline
	1 & β\_0 & 3.76337 & 3.88215e10 \\
	2 & β\_l & 0.628663 & 0.19689 \\
	3 & β\_k & 0.386908 & 0.0865049 \\
\end{tabular}


14

----

**Question 2.9**



From the demand equation alone we can not separately identify $\eta$ from $p_{t}$. To see it, take logs to get:

$q_{j,t}= d_t + \eta p_{j,t}+ \xi_{j,t}$

Time fixed effects remove $d_t$ but we can not separate $\eta p_{j,t}$ -i.e we could make an assumption on the shock, but we would recover always a composition of the two, as prices are unobserved. One way to solve this is follow De Loecker/Wacynsky? and recover prices from markup. In this case, we don't observe the cost of input, so their method does not work. Another possibility, is to do the following:


First, let's define the production function we are estimating with the data: 

$$
y_{jt}= \beta_{0} + \beta_{l}l_{j,t}+\beta_{k}k_{j,t}+\omega_{j,t} + \epsilon_{j,t}
$$

We know that $y_{j,t}=\log(Q_{j,t}P_{j,t})=q_{j,t} + p_{j,t}$

Now with the demand model we suppose, after taking logs, that $q_{j,t}= d_t + \eta p_{j,t}+ \xi_{j,t}$

If we isolate price we have: $p_{j,t}= \frac{1}{\eta}(q_{j,t}- d_t \xi_{j,t})$

Then we can plug in this to the previous expression:

$y_{j,t}=q_{j,t} + \frac{1}{\eta}(q_{j,t}- d_t \xi_{j,t})$
$y_{j,t}= (1 + \frac{1}{\eta})q_{j,t} + \frac{1}{\eta}(- d_t \xi_{j,t})$

Thus, what we are estimating in reality is 
$$
y_{j,t}= \beta_{0} + \frac{1}{\eta} d_t + \kappa \beta_{l}l_{j,t} + \kappa \beta_{k}k_{j,t} + \kappa \omega_{j,t} + \text{error}
$$

Where the estimated $\hat{\beta_{l}}=\kappa \beta_{l} $ where $\kappa= (1+ \frac{1}{\eta})$. One way to identify $\eta$ in this case is to assume CRS: 

$$
\beta_{l}+ \beta_{k}=1
$$
Thus 

$$
\hat{\beta_{l}}+ \hat{\beta_{k}}= \hat{\kappa}
$$

And we can get:

$$
\eta= \frac{1}{\hat{\kappa}-1}
$$

REVISE!!!!
**This needs to be revised; not entirely sure what is the right solution if we don't observe wages. This is not correct because coefficents are biased, no? so we are not estimating the correct object.** 


